# An example of using the python eTaSL driver in a python notebook (2)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML

#%load_ext autoreload
#%autoreload 2

In [ ]:
# change the width of the notebook:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
from etasl_py.etasl import  etasl,to_rad,to_deg,integrate,array_to_dict,dict_to_array
import math
e = etasl(nWSR=300, cputime=10, regularization_factor=1E-3)
e.readTaskSpecificationFile("etasl_robot_ur10.lua")
e.readTaskSpecificationFile("etasl_simple_task.lua")

# Simulating the eTaSL controller using the high-level eTaSL driver

In [ ]:
from etasl_py.etasl import  etasl,to_rad,to_deg,integrate,array_to_dict,dict_to_array
class etasl_simulator:
    def __init__(self, 
                    nWSR                  = 300,
                    cputime               = 1000,
                    regularization_factor = 1E-4
                ):
        self.nWSR=nWSR
        self.cputime=cputime
        self.regularization_factor = regularization_factor
        self.input_cb     = []
        self.output_cb    = []
        self.etasl  = etasl(self.nWSR,self.cputime,self.regularization_factor)
        
    def initialize(self,initial_jpos,max_time=3,time_step=0.05,convergence_crit=1E-4):
        self.initial_jpos = initial_jpos
        self.initial_jpos = self.etasl.initialize(initial_jpos,max_time=3,time_step=0.05,convergence_crit=1E-4)
        
    def __str__(self):
        return """etasl_simulator(
                     WSR : {}
                     cputime : {}
                     regularization_factor : {}
                     # input callbacks registered : {}
                     # output callbacks registered : {}
                  )""".format(
                            self.nWSR, 
                            self.cputime, 
                            self.regularization_factor,
                            len(self.input_cb),
                            len(self.output_cb)
                    ) 
    def getInputNames(self):
        return self.etasl.getInputNames()
    def getOutputNames(self):
        return self.etasl.getOutputNames()
    def getVariables(self,flag):
        return self.etasl.getVariables(flag)
    def nrOfBoxConstraints(self):
        return self.etasl.nrOfBoxConstraints()
    def nrOfScalarConstraints(self):
        return self.etasl.nrOfScalarConstraints()
    def readTaskSpecificationFile(self,name):
        return self.etasl.readTaskSpecificationFile(name)
    def readTaskSpecificationString(self,estr):
        return self.etasl.readTaskSpecificationString(estr)
    def printContext(self):
        return self.etasl.printContext()
    # copy doc from underlying object for a given set of methods:
    __init__.__doc__ = etasl.__init__.__doc__
    readTaskSpecificationFile.__doc__=etasl.readTaskSpecificationFile.__doc__
    getInputNames.__doc__ = etasl.getInputNames.__doc__
    getOutputNames.__doc__ = etasl.getOutputNames.__doc__
    nrOfBoxConstraints.__doc__=etasl.nrOfBoxConstraints.__doc__
    nrOfScalarConstraints.__doc__ = etasl.nrOfScalarConstraints.__doc__
    readTaskSpecificationString.__doc__ = etasl.readTaskSpecificationString.__doc__
    printContext.__doc__ = etasl.printContext.__doc__
    
    #def add_input_cb(self, f ):
    #    self.input_cb.append(f);
    #def add_output_cb(self, f ):
    #    self.output_cb.append(f);    
        
    def setInputTable(self, inptable, inpveltable=None):
        '''
        sets input for the simulation,
        
        input:
            inptable:  pandas dataframe with input variables, should have N rows (see simulate) and
                       the labels should correspond to the input variables
            veltable:  OPTIONAL: pandas dataframe with the velocity for the input variables
        '''
        self.INP = inptable
        self.INPVEL = inpveltable
        
    def simulate(self, N, dt):
        self.TIME   = np.arange(0,N)*dt
        LBL         = [v for v in self.getVariables(3)];
        LBL.append('time')
        self.POS    = pd.DataFrame(np.zeros((len(self.TIME),len(LBL))),columns=LBL)
        self.VEL    = pd.DataFrame(np.zeros((len(self.TIME),len(LBL))),columns=LBL)
        OLBL        = self.etasl.getOutputNames()
        self.OUTP   = pd.DataFrame(np.zeros((len(self.TIME),len(OLBL))), columns=OLBL)
        pos         = {v:0.0 for v in self.getVariables(3)};
        for k,v in initial_jpos.iteritems():
            pos[k]=v
        pos["time"] = self.TIME[0] 
        i =0
        while True:
            self.etasl.setInput(self.INP.iloc[i,:].to_dict())
            if not self.INPVEL is None:
                self.etasl.setInputVelocity(self.INPVEL.iloc[i,:].to_dict())
            self.etasl.setJointPos(pos)
            self.etasl.solve()
            vel = self.etasl.getJointVel(3)
            #print("pos")
            #print(pos)
            #print("vel")
            #print(vel)
            #print("dt")
            #print(dt)
            pos = integrate(pos, vel, dt)
            self.OUTP.iloc[i] = self.etasl.getOutput()
            self.POS.iloc[i] = pos
            self.VEL.iloc[i] = vel
            i    = i + 1
            if i >= len(self.TIME):
                break
    




#def cb_table_input(df):
#    '''returns a callback function that uses all columns of the given data 
#       frame to fill in input values for eTaSL'''
#    def cb( e, seqnr,time):
#        '''callback that sets the input based on a table and the given seqnr'''
#        e.setInput(df.iloc[seqnr,:].to_dict())
#        return
#            
#    return cb

#def cb_table_output(df):
#    def cb(e, seqnr,time):
#        '''callback that stores output in a a table at the given seqnr'''
#        row=e.getOutput()
#        row['time'] = time
#        df.iloc[seqnr,:] =  row
#        return
#    return cb

In [ ]:
e = etasl_simulator(regularization_factor= 0.001)
e.readTaskSpecificationFile("etasl_robot_ur10.lua")
e.readTaskSpecificationFile("etasl_simple_task.lua")
initial_jpos = {'shoulder_pan_joint':0,
                'shoulder_lift_joint':0,
                'elbow_joint':0,  
                'wrist_1_joint':0, 
                'wrist_2_joint':0, 
                'wrist_3_joint':0}




N=2000
dt=0.01
time = np.arange(0,N)*dt
inp=pd.DataFrame()
#inp_lbl=['global.tgt_x', 'global.tgt_y', 'global.tgt_z']
#inp=pd.DataFrame( np.zeros((len(time),len(inp_lbl))),  columns=inp_lbl)
inp['global.tgt_x'] = time*0
inp['global.tgt_y'] = np.sin(time)*0.15 - 0.7
inp['global.tgt_z'] = time*0 + 0.3
#inpvel=pd.DataFrame( np.zeros((len(time),len(inp_lbl))),  columns=inp_lbl)
inpvel = pd.DataFrame()
inpvel['global.tgt_x'] = time*0
inpvel['global.tgt_y'] = np.cos(time)*0.15
inpvel['global.tgt_z'] = time*0

e.setInputTable(inp,inpvel)
e.initialize(initial_jpos)
e.simulate(N=N,dt=dt)

In [ ]:
idx=range(1,3)
HTML( "<br><b>e.OUTP:</b> "+e.OUTP.iloc[idx].to_html() + 
      "<br><b>e.INP:</b>  "+e.INP.iloc[idx].to_html()  +
      "<br><b>e.INPVEL:</b>  "+e.INPVEL.iloc[idx].to_html()  +
      "<br><b>e.POS:</b>  "+e.POS.iloc[idx].to_html()  +
      "<br><b>e.VEL:</b>  "+e.VEL.iloc[idx].to_html() )

In [ ]:
plotw=24
w=4;h=2
plt.figure(figsize=(plotw,9))
for i in range(e.POS.shape[1]):
    ax=plt.subplot(h,w,i+1)
    ax.plot(e.TIME, e.POS[e.POS.columns[i]]*180.0/np.pi)
    plt.ylabel(e.POS.columns[i] +" [deg]")
    plt.xlabel('time [s]')
w=4;h=2
plt.figure(figsize=(plotw,9))
for i in range(e.VEL.shape[1]):
    ax=plt.subplot(h,w,i+1)
    ax.plot(e.TIME, e.VEL[e.VEL.columns[i]])
    plt.ylabel(e.VEL.columns[i] +" [rad/s]")
    plt.xlabel('time [s]')

w=4;h=1;
plt.figure(figsize=(plotw,4))
for i in range(e.INP.shape[1]):
    plt.subplot(h,w,i+1)
    plt.plot(e.TIME, e.INP[e.INP.columns[i]])
    plt.ylabel(e.INP.columns[i])
    plt.xlabel('time [s]')
    
w=4;h=1;
plt.figure(figsize=(plotw,4))
for i in range(e.OUTP.shape[1]):
    plt.subplot(h,w,i+1)
    plt.plot(e.TIME, e.OUTP[e.OUTP.columns[i]])
    plt.ylabel(e.OUTP.columns[i])
    plt.xlabel('time [s]')    
    
    
plt.figure(figsize=(plotw,4))
plt.subplot(1,3,1)
plt.plot(e.TIME, e.OUTP['global.ee_x'] - e.INP['global.tgt_x'])
plt.xlabel('time [s]')
plt.ylabel('error x')

plt.subplot(1,3,2)
plt.plot(e.TIME, e.OUTP['global.ee_y'] - e.INP['global.tgt_y'])
plt.xlabel('time [s]')
plt.ylabel('error y')

plt.subplot(1,3,3)
plt.plot(e.TIME, e.OUTP['global.ee_z'] - e.INP['global.tgt_z'])
plt.xlabel('time [s]')
plt.ylabel('error z')

In [ ]:
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.layouts import gridplot

pw=500
ph=400

# output to static HTML file
output_notebook()
p1 = figure(x_axis_label='time', 
            y_axis_label= "error x",
            plot_width=pw,
            plot_height=ph)
p1.line(e.TIME, e.OUTP['global.ee_x'] - e.INP['global.tgt_x'], line_width=2)

p2 = figure(x_axis_label='time', 
            y_axis_label= "error y",
            plot_width=pw,
            plot_height=ph,
            x_range = p1.x_range)
p2.line(e.TIME, e.OUTP['global.ee_y'] - e.INP['global.tgt_y'], line_width=2)

p3 = figure(x_axis_label='time', 
            y_axis_label= "error x",
            plot_width=pw,
            plot_height=ph,
            x_range = p1.x_range)
p3.line(e.TIME, e.OUTP['global.ee_z'] - e.INP['global.tgt_z'], line_width=2)

pg = gridplot( [ [p1,p2, p3 ] ])    
show(pg)

In [ ]:




def cb_table_input(df):
    '''returns a callback function that uses all columns of the given data 
       frame to fill in input values for eTaSL'''
    def cb( e, seqnr,time):
        '''callback that sets the input based on a table and the given seqnr'''
        e.setInput(df.iloc[seqnr])
        return
            
    return cb

def cb_table_output(df):
    def cb(e, seqnr,time):
        '''callback that stores output in a a table at the given seqnr'''
        row=e.getOutput()
        row['TimeStamp'] = time
        df.iloc[seqnr] = e.row
        return
    return cb
        

## 2. Simulating the eTaSL controller using the python eTaSL driver

### 2.1 Define the inputs to the controller

In [ ]:
# define the time period for simulation:
T    = 20
dt   = 0.01
time = np.arange(0,T,dt)

# define INPUT (and its time derivative)
INP_LBL = ['tgt_x','tgt_y','tgt_z']  # corresponds to eTaSL definition
INP      = np.zeros((len(time),3))
INP[:,0] = time*0
INP[:,1] = np.sin(time)*0.15 - 0.7
INP[:,2] = time*0 + 0.3

INPVEL   = np.zeros((len(time),3))
INPVEL[:,0] = time*0
INPVEL[:,1] = np.cos(time)*0.15
INPVEL[:,2] = time*0


In [ ]:
print (e.getOutputNames())

### 2.2 define the state (both robot- and feature variables) and output arrays

These are the arrays that will contain the history of the state variables for both robot- and feature variables, and the output.

In [ ]:
# robot variables:
J_LBL = [ 'shoulder_pan_joint',
          'shoulder_lift_joint', 
          'elbow_joint', 
          'wrist_1_joint',
          'wrist_2_joint',
          'wrist_3_joint']
JPOS     = np.zeros((len(time),6))
JVEL     = np.zeros((len(time),6))

# feature variables:
F_LBL = ['f1']
FPOS  = np.zeros((len(time),1))
FVEL  = np.zeros((len(time),1))

# output expressions:
OUTP_LBL = ['global.ee_x','global.ee_y','global.ee_z']
OUTP     = np.zeros((len(time),3))


In [ ]:
from etasl_py.etasl import  etasl,to_rad,to_deg,integrate,array_to_dict,dict_to_array
import math
import numpy as np


e = etasl(nWSR=300, cputime=10, regularization_factor=1E-4)
e.readTaskSpecificationFile("etasl_robot_ur10.lua")
e.readTaskSpecificationFile("etasl_simple_task.lua")

#initial_jpos = to_rad({'q1':0, 'q2':30,'q3':0,'q4':60,'q5':0,'q6':90,'q7':0})
initial_jpos = to_rad({'elbow_joint': 0, 
                         'shoulder_pan_joint': 0, 
                         'wrist_3_joint': 0, 
                         'wrist_1_joint': 0, 
                         'shoulder_lift_joint': 0, 
                         'wrist_2_joint': 0})

converged_val = e.initialize(initial_jpos,max_time=3,time_step=0.05,convergence_crit=1E-4)
jpos = initial_jpos

jpos["time"] = time[0] 

i =0
fpos     = {'f1':0.0}
while True:
    e.setInput(array_to_dict(INP[i,:],INP_LBL))
    e.setInputVelocity(array_to_dict(INPVEL[i,:],INP_LBL))
    
    e.setJointPos(jpos)
    e.setJointPos(fpos)
    e.solve()
    jvel = e.getJointVel(1)
    fvel = e.getJointVel(2)
    jpos = integrate(jpos, jvel, dt)
    fpos = integrate(fpos, fvel, dt)
    outp = e.getOutput()
    OUTP[i,:] = dict_to_array(outp, OUTP_LBL)

    JPOS[i,:] = dict_to_array(jpos, J_LBL)
    JVEL[i,:] = dict_to_array(jvel, J_LBL)
    FPOS[i,:] = dict_to_array(fpos, F_LBL)
    FVEL[i,:] = dict_to_array(fvel, F_LBL)
    i    = i + 1
    if i >= len(time):
        break
    #print(outp)
    

In [ ]:
e.getVariables(3)

In [ ]:
print(converged_val)


### Plot the result of the simulation:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(1,figsize=(14,4))
plt.subplot(1,2,1)
plt.plot(time,JPOS*180.0/np.pi)
plt.title('joint positions [DEG]')
plt.xlabel("time [s]")

plt.subplot(1,2,2)
plt.plot(time,JVEL)
plt.title('joint velocities [RAD/s]')
plt.xlabel("time [s]")

plt.figure(2)
plt.plot(time,OUTP,'b')
plt.plot(time,INP,'r')
plt.title('end effector and target position')
plt.xlabel("time [s]")


plt.figure(3)
plt.plot(time,FPOS)
plt.title('feature variables')
plt.xlabel("time [s]")



plt.figure(4)
plt.plot(time,OUTP-INP)
plt.title('control error')
plt.xlabel("time [s]")
#plt.ylim(-1E-3,1E-3)
plt.show()

In [ ]:
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.layouts import gridplot

# output to static HTML file
output_notebook()

# create a new plot with a title and axis labels
p = figure(title="simple line example", x_axis_label='time', y_axis_label='axis 3')
p.plot_width=800
p.plot_height=400
# add a line renderer with legend and line thickness
p.line(time, JVEL[:,1], legend="axis 3", line_width=2)

# show the results
show(p)
p = figure(title="control error", x_axis_label='time', y_axis_label='axis 3')
p.plot_width=800
p.plot_height=400
# add a line renderer with legend and line thickness
p.line(time, OUTP[:,1]-INP[:,1], line_width=2)

# show the results
show(p)

In [ ]:
time